In [1]:
import pandas as pd
import numpy as np
import sqlite3

In [2]:
conn = sqlite3.connect('../creig-server-1/prisma/test.db')

In [3]:
jobs = pd.read_sql('SELECT InstantJobs.*, JobType.name as category, Location.name as location FROM InstantJobs JOIN JobType ON InstantJobs.typeId = JobType.id JOIN Location ON InstantJobs.locationId = Location.id',conn)
jobs.head()

,id,title,description,typeId,locationId,rateId,creatorId,createdAt,updatedAt,category,location
0,bed0d85e-7a34-44ad-bb8d-2c63f761347b,Graphic Designer Needed,Need to design a company logo and branding mat...,8a710054-537a-4de0-afa5-44533319dbb6,d9950931-4bc9-4734-8fe8-1c88b0ab0c09,b5542965-465c-4423-8337-6e378c6154de,daac0b8b-0f92-41ae-a854-5dc6aa7a2275,1719815937375,1719815937375,Consulting,Pokhara
1,8e10a97b-cefc-4173-86d7-a2078d0d1901,Website Developer Required,Need to develop a responsive website for an e-...,a6e31aac-fc9e-4401-a333-05c9e894553b,d5edb1d1-bae5-4ad9-b71d-5c56d296b840,dd7808ed-638f-4e61-95e1-4a12355f4f24,d8a60c17-e975-48ec-984d-7b5601d9108c,1719815949259,1719815949259,Handyman Services,Hetauda
2,30a3b2ee-3194-4df7-aa0d-3b388e3a7e1f,Content Writer Needed,Need to write SEO optimized blog posts on vari...,46aed486-88ce-405b-847d-eff95e7c5169,d9950931-4bc9-4734-8fe8-1c88b0ab0c09,58bafc5b-efc6-4f67-9707-1d79090b9dae,e8c8c614-b006-4ce1-861d-a6ad18622361,1719815962114,1719815962114,Courier Services,Pokhara
3,b2906902-32e2-4cbd-8d86-529b094fd523,Photographer Needed for Event,Need to capture high-quality photos at a corpo...,a72a7baf-86c2-48d3-9f4b-3fbca75e80ed,d5edb1d1-bae5-4ad9-b71d-5c56d296b840,1ee95200-743f-46c4-b242-076d6f0052be,f49509f8-fd01-4481-b6a8-e2d4aac290ad,1719816032172,1719816032172,Digital Marketing,Hetauda
4,4aae55e9-739e-473a-8ea5-1887061f6d7b,Video Editor Required,Need to edit promotional videos for social med...,0546e60d-4b3f-4f37-80f3-2d4aef2bd3fb,829f45ed-5a75-4740-ad9c-919f5c245824,bbd83d97-a01d-4bd7-b96c-5a3042bf4eca,edff522c-f1e2-4293-a968-67d78759620c,1719816048456,1719816048456,Pet Sitting,Itahari


In [4]:
jobs["description"] = jobs["description"].apply(lambda x:x.split())

In [5]:
jobs['category'] = jobs['category'].apply(lambda x: ["".join(x)])

In [6]:
jobs['location'] = jobs['location'].apply(lambda x: ["".join(x)])

In [7]:
jobs["tags"] = jobs["description"] + jobs["category"] + jobs["location"]

In [8]:
new_jobs = jobs[["id","title","tags"]]
new_jobs.dropna(inplace=True)
new_jobs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      33 non-null     object
 1   title   33 non-null     object
 2   tags    33 non-null     object
dtypes: object(3)
memory usage: 924.0+ bytes


/tmp/ipykernel_49912/1230452298.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_jobs.dropna(inplace=True)


In [9]:
from sklearn.feature_extraction.text import CountVectorizer

In [10]:
cv = CountVectorizer(max_features = 100, stop_words="english")

In [11]:
new_jobs["tags"] = new_jobs['tags'].apply(lambda x:' '.join(x))

/tmp/ipykernel_49912/677792311.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_jobs["tags"] = new_jobs['tags'].apply(lambda x:' '.join(x))


In [12]:
cv.fit_transform(new_jobs["tags"]).toarray().shape

(33, 100)

In [13]:
vectors = cv.fit_transform(new_jobs['tags']).toarray()

In [14]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [15]:
def stem(text):
  y=[]
  for i in text.split():
    y.append(ps.stem(i))
  return " ".join(y)

In [16]:
new_jobs['tags'] = new_jobs["tags"].apply(stem)

/tmp/ipykernel_49912/1727861123.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_jobs['tags'] = new_jobs["tags"].apply(stem)


In [17]:
from sklearn.metrics.pairwise import cosine_similarity

In [18]:
cosine_similarity(vectors)

array([[1.        , 0.1767767 , 0.30151134, ..., 0.33333333, 0.25      ,
        0.16666667],
       [0.1767767 , 1.        , 0.21320072, ..., 0.11785113, 0.1767767 ,
        0.11785113],
       [0.30151134, 0.21320072, 1.        , ..., 0.20100756, 0.15075567,
        0.10050378],
       ...,
       [0.33333333, 0.11785113, 0.20100756, ..., 1.        , 0.16666667,
        0.11111111],
       [0.25      , 0.1767767 , 0.15075567, ..., 0.16666667, 1.        ,
        0.16666667],
       [0.16666667, 0.11785113, 0.10050378, ..., 0.11111111, 0.16666667,
        1.        ]])

In [19]:
similarity = cosine_similarity(vectors)

In [20]:
def recommend(job):
    # Check if the job exists in the new_jobs DataFrame
    if job not in new_jobs['id'].values:
        print("Job ID not found.")
        return

    # Get the index of the job
    job_index = new_jobs[new_jobs['id'] == job].index[0]

    # Calculate distances
    distances = similarity[job_index]

    # Sort jobs based on distance and get top 5 similar jobs
    jobs_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]

    # Print job IDs of the recommended jobs
    for i in jobs_list:
        print(new_jobs.iloc[i[0]].id)

In [22]:
recommend('8e10a97b-cefc-4173-86d7-a2078d0d1901')

9f64baa0-f96b-4e48-97bf-dad8ce7a039c
4a3a0ed2-1c81-4e00-b3aa-8d83fdcb4f23
87e2e566-1b5f-4d36-a526-64eb43353c7d
fd99c630-2261-4546-bbc8-db72d73da615
67fc3875-8595-476e-9c7c-ab43d9ddae22
